Do rozwiązania problemu zastosowano algorytm genetyczny. Pojedyncze rozwiązanie (genom) reprezentowane jest przez wektor (listę) wymiaru $n$ w nastepujący sposób: numer indeksu oznacza numer placu budowy, a liczba odpowiadająca $i$-temu indeksowi oznacza numer budynku przypisany do $i$-tego placu budowy. Do selekcji genomów zastosowano metodę selekcji turniejowej (niedeterministycznej - losujemy grupy w obrębie których następnie ustalamy ranking). Wprowadzano dodatkowo element strategii elitarnej: zachowywany jest najlepszy osobnik w populacji bieżącej (w postaci niezmienionej, niepoddawany procesom mutacji ani krzyżowania). Mutacje w obrebie populacji zachodzą w sposób losowy: losowane są osobniki (z zadanym prawdopoodbieństwen, którego domyślna wartość wynosi 0.5), które podlegają mutacji. Mutacja polega na wylosowaniu dwóch indeksów, które następnie są zamieniane (permutowane) ze sobą. 
Proces krzyżowania przebiega w następujacy sposob: losujemy dwa osobniki (i usuwamy je z listy zawierającej populację), a następnie losowany jest przedział (losujemy dwa indeksy, mniejszy z nich stanowi początek przedziału, a większy - koniec przedziału). W oparciu o wylosowany przedział tworzymy osobniki potomne według nastepującego schematu (obszar zaznaczony na zielono, to nasz wylosowany przedział):

<img src = "https://uploadfile.pl/pobierz/2242177---0gqx/schemat.png" />


In [14]:
import random
import numpy as np

class Puzzle:
 
 def __init__(self, filepath):
             
  file = open(filepath, "r")
  #lines = file.split('\n')
 
  with open(filepath) as file:
    lines = []
    with open(filepath) as file:
      lines = [line.rstrip() for line in file]


  self.n = int(lines[0])
  i=1
  while(len(lines[i]) < self.n):
        i=i+1
       
  self.A = [[int(x) for x in lines[i+j].split()] for j in range(self.n)]
  
  i=i + self.n
 
  while(len(lines[i])<self.n):
        i=i+1
        
  self.D = [[int(x) for x in lines[i+j].split()] for j in range(self.n)]
  

      
class GeneticSolution:
    
#Pojedyncze osobniki sa reprezentowane przez macierze wymiaru nxn, takie, ze 1 wystepuje na wspolrzednej (i,j) 
#wtedy i tylko wtedy gdy na i-tym placu budowy stawiamy j-ty budynek
    
  def __init__(self, n_popul, puzzle):   #Tworzenie populacji poczatkowej
      
      self.n = puzzle.n
      self.A = puzzle.A
      self.D = puzzle.D
      self.population = []
      for p in range(n_popul):
         x = list(range(self.n))
         random.shuffle(x) 
         genom = x
         self.population.append(genom)
      self.numberpopul = n_popul
        
        

  def CalculateTotalErrors(self):
      Errors = []
            
      for genom in self.population:
      
          result = 0
     
          for i in range(self.n):
              for j in range(self.n):
                              
                  result = result + self.A[int(genom[i])][int(genom[j])]*self.D[i][j]
              
          Errors.append(result)
      return Errors
            
          
           
  def Selection (self, tournament_size):       #dokonuje ewaluacji osobnikow i selekcji
      
      fun_values = self.CalculateTotalErrors()     
     
      best_value =  min(fun_values)
              
      best_genom_ind = fun_values.index(best_value)
      best_genom = self.population[best_genom_ind]
      #wprowadzamy element strategii elitarnej: zachowujemy najlepszego osobnika w populacji biezacej
    
      newpopulation = []
               
      for i in range(self.numberpopul):  #dokonujemy selekcji metoda turniejowa
          S = []
          for j in range(tournament_size):
              ind = int(random.random()*self.numberpopul)
              S.append(ind)
          ranking = [fun_values[ind] for ind in S]
          winner_ind = S[ranking.index(min(ranking))]
          newpopulation.append( self.population[winner_ind])
         
         
      self.population = newpopulation
             
      return best_genom
  
  def Crossover(self):
    
    new_generation = []
    for i in range(int(self.numberpopul/2)):
       parents = []
       rand_ind1 = int(random.random()*len(self.population))
       parent1 = self.population[rand_ind1]
       del self.population[rand_ind1]
       parents.append(parent1)
       rand_ind2 = int(random.random()*len(self.population))
       parent2 = self.population[rand_ind2]
       del self.population[rand_ind2]
       parents.append(parent2)
       pointa = int(random.random()*(self.n -2))
       pointb = int(random.random()*(self.n -2))
       while pointb==pointa:
           pointb= int(random.random()*(self.n -2))
       point1 = min(pointa, pointb)
       point2 = max(pointa, pointb)
       
       offspring_a = np.zeros(self.n)
       offspring_a[point1:point2] = parents[1][point1:point2]
       already_used_genes = set(parents[1][point1:point2])
       i = 0
       p = 0
       while i<point1 and p<self.n:
           if parents[0][p] not in already_used_genes:
               offspring_a[i] = parents[0][p]
               already_used_genes.add(parents[0][p])
               i+=1
          
           p+=1
           
       j = point2 + 1
       while j<self.n and p<self.n:
           if parents[0][p] not in already_used_genes:
               offspring_a[j] = parents[0][p]
               already_used_genes.add(parents[0][p])
               j+=1
           
           p+=1
      
       new_generation.append(offspring_a)
       
           
       offspring_b = np.zeros(self.n)
       offspring_b[point1:point2] = parents[0][point1:point2]
       already_used_genes = set(parents[0][point1:point2])
       i = 0
       p = 0
       while i<point1 and p<self.n:
           if parents[1][p] not in already_used_genes:
               offspring_b[i] = parents[1][p]
               already_used_genes.add(parents[1][p])
               i+=1
          
           p+=1
           
       j = point2 + 1
       while j<self.n and p<self.n:
           if parents[1][p] not in already_used_genes:
               offspring_b[j] = parents[1][p]
               already_used_genes.add(parents[1][p])
               j+=1
           
           p+=1
       
       new_generation.append(offspring_b)
       
       self.population = new_generation
       self.numberpopul = len(new_generation)
                   
                   
    
  def Mutation( self, probability = 0.5):
    
    for i in range( self.numberpopul -1):
       if random.random() < probability:
              inds = random.choices(range(self.n), k=2)
              mut = self.population[i]
              pom = mut[inds[0]]
              mut[inds[0]] = mut[inds[1]]
              mut[inds[1]] = pom
              self.population[i] = mut
             
                 
             
                           

def Solve(filepath, n_popul, tournament_size, n_iter, mutation_probability = 0.5):
    puzzle = Puzzle(filepath)
    genetic = GeneticSolution(n_popul, puzzle)
    
    for i in range(n_iter):
        
       best_genom = genetic.Selection(tournament_size)
       genetic.Crossover()
       genetic.Mutation(mutation_probability)
       genetic.population.append(best_genom)
       genetic.numberpopul +=1
       
    func_values = genetic.CalculateTotalErrors()     
    best_value =  min(func_values)
    best_genom_ind = func_values.index(best_value)
    best_genom = genetic.population[best_genom_ind]
              
    return best_genom, best_value
 

In [15]:
Solve('task2.txt', 1000, 2, 100)


([7,
  15,
  31,
  24,
  29,
  3,
  23,
  0,
  4,
  8,
  16,
  20,
  30,
  13,
  25,
  28,
  5,
  21,
  10,
  26,
  11,
  12,
  19,
  6,
  14,
  2,
  1,
  22,
  18,
  27,
  17,
  9],
 24526)

In [16]:
Solve('task2.txt', 1000, 2, 1000)

(array([11., 10., 13., 27., 21., 12.,  2., 30., 20.,  7., 15.,  3., 26.,
         6.,  8.,  9., 23., 24.,  1., 31., 28., 16., 25., 29., 22.,  0.,
         0.,  5., 17., 18., 19., 14.]),
 22742)

In [23]:
Solve('task2.txt', 100000, 2, 10000)


([26,
  17,
  29,
  30,
  14,
  5,
  15,
  13,
  27,
  3,
  25,
  24,
  8,
  9,
  12,
  31,
  16,
  7,
  4,
  28,
  10,
  2,
  1,
  20,
  21,
  19,
  22,
  6,
  11,
  18,
  0,
  23],
 22912)

In [28]:
Solve("newtask1", 10000, 2, 10000)

(array([16., 17., 18., 13., 11.,  8.,  4.,  1.,  9., 10.,  0.,  0.,  6.,
        14.,  0.,  0.,  7., 15.,  5.]),
 15892770)

In [29]:
Solve("newtask1", 100000, 2, 10000)


(array([16., 17., 18.,  8., 11.,  5., 14.,  1., 12., 13.,  9.,  4.,  6.,
         0.,  0.,  0.,  7.,  3., 15.]),
 16920674)

In [31]:
Solve("ntask3", 10000, 2, 10000)


(array([83., 26., 58., 56.,  8., 54., 28., 31., 37., 75., 15., 55., 33.,
        60., 12., 17., 88., 44., 38., 20.,  1., 81.,  4., 66., 78., 62.,
        77., 76., 41., 36., 87., 53., 14., 57., 22., 46., 47.,  5., 50.,
        18., 71.,  7., 65., 69., 85., 21., 43., 52., 30., 61.,  3., 67.,
        84., 80., 32., 24., 49., 74., 40.,  6., 25., 23., 72., 64., 89.,
        10., 35., 82., 34., 79., 27., 63., 59.,  9., 51., 42., 45., 19.,
         2., 86.,  0., 39., 16., 13., 29., 70., 68.,  0.,  0.,  0.]),
 15843229)